To save and restore your variables, all you need to do is to call the **tf.train.Saver()** at the end of you graph.

# 1. Save and restore two variables

### 1.1 Save

In [1]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.__version__

'1.13.1'

In [2]:
#tf.reset_default_graph()
a = tf.get_variable('A',initializer=tf.constant(3, shape=[2]))
b = tf.get_variable('B',initializer=tf.constant(5, shape=[3]))

# varible need to be initilize before use
init_op = tf.global_variables_initializer()

with tf.Session() as ss:
    ss.run(init_op)
    a_out,b_out = ss.run([a,b])
    print('a=',a_out)
    print('b=',b_out)

Instructions for updating:
Colocations handled automatically by placer.
a= [3 3]
b= [5 5 5]


All of the variables exist in the scope of the session. So, after the session is closed, we will loose the variable.

In order to save the variable, we will call the saver function using _tf.train.Saver()_ in our graph. This function will find all the variables in the graph. We can see the list of all variables in  _ _var_list._ Let's create a saver object and take a look at the _ _var_list_ in the object:

In [3]:
# create saver object
saver = tf.train.Saver()
for i ,var in enumerate(saver._var_list):
    print(f'var {i} : {var}')

var 0 : <tf.Variable 'A:0' shape=(2,) dtype=int32_ref>
var 1 : <tf.Variable 'B:0' shape=(3,) dtype=int32_ref>


In [4]:
# Now we save this variable
#            path/<file_names>
path = './savers/saved_variable'
with tf.Session() as ss:
    ss.run(init_op)
    # save the session in the disk
    saved_path = saver.save(ss,path)
    print(f'model save at {saved_path}')

model save at ./savers/saved_variable


In [5]:
# check the files
import os
for file in os.listdir('./savers'):
    print(file)

checkpoint
saved_variable.data-00000-of-00001
saved_variable.index
saved_variable.meta


**.data:** Contains variable values

**.meta:** Contains graph structure

**.index:** Identifies checkpoints

### 1.2 Restore
by using saver.restore() we can load the saved variables into a session

In [6]:
with tf.Session() as ss:
    # restore the saved varible into ss session fron path
    saver.restore(ss,path) # all varaibles load into this session
    a_out, b_out = ss.run([a,b])
    print(f'a = {a_out}')
    print(f'b = {b_out}')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./savers/saved_variable
a = [3 3]
b = [5 5 5]


Note:: notice that this time we did not initialize the variables in our session. Instead we restored them from the disk

In order to restore the parameters, the graph should be defined.Since we defined the graph in top, we didn't have a problem restoring the parameters. But what happens if we have not loaded the graph??

In [7]:
# delete the current_graph
tf.reset_default_graph()
try:
    with tf.Session() as ss:
        saver.restore(ss,path)
        a_out, b_out = ss.run([a,b])
        print(f'a = {a_out}')
        print(f'b = {b_out}')
except Exception as e:
    print(str(e))

INFO:tensorflow:Restoring parameters from ./savers/saved_variable
The Session graph is empty.  Add operations to the graph before calling run().


In [9]:
# Session is empty?

so what happen here? In tensorflow graph we can access variables in only active sessions. If we want to access variable outer the session then we can store them into another varible either or save into a file. Now, here we reset the graph. i.e graph is empty it cannot access the varible until we dont provide the structure of graph

 we can define the graph in two ways:
### 1.2.1 Define the graph from scratch and then run the session

what you can do is to create the graph and then restore your variables

In [10]:
tf.reset_default_graph()

# create variable a and b
a = tf.get_variable(name = 'A',initializer=tf.constant(3,shape=[2]))
b = tf.get_variable(name = 'B',initializer=tf.constant(5,shape=[3]))

# init
init_op = tf.global_variables_initializer()

# create saver object    
saver = tf.train.Saver() 
 
# run the session
with tf.Session() as ss:
    saver.restore(ss,path)
    ao,bo = ss.run([a,b])
    print(ao,bo)

INFO:tensorflow:Restoring parameters from ./savers/saved_variable
[3 3] [5 5 5]


What happen if we don't know the graph we are using someone else's pre-trained model?

### 1.2.2. Restore the graph from .meta file

When we save the variables, it creates a .meta file. This file contains the graph structure. Therefore, we can import the meta graph using tf.train.import_meta_graph() and restore the values of the graph. Let's import the graph and see all tensors in the graph:

In [13]:
# delete the current graph
tf.reset_default_graph()

import_graph = tf.train.import_meta_graph('./savers/saved_variable.meta')

# list all the tensors in the graph
for tensor in tf.get_default_graph().get_operations():
    print(tensor.name)

Const
A
A/Assign
A/read
Const_1
B
B/Assign
B/read
init
save/filename/input
save/filename
save/Const
save/SaveV2/tensor_names
save/SaveV2/shape_and_slices
save/SaveV2
save/control_dependency
save/RestoreV2/tensor_names
save/RestoreV2/shape_and_slices
save/RestoreV2
save/Assign
save/Assign_1
save/restore_all


In [16]:
with tf.Session() as ss:
    # restore the saved variable
    import_graph.restore(ss, path)
    ao,bo = ss.run(['A:0','B:0'])
    print(ao,bo)

INFO:tensorflow:Restoring parameters from ./savers/saved_variable
[3 3] [5 5 5]


Why I don't use a,b in run? because a,b saved with A:0 and B:0 names in .meta file